# 🚀 **Passo 1: Preparando os Dados e Definindo o Que Queremos**

## **Aula 1.1: Entendendo o Problema**

---

### **Tá, mas o que é migração de modelo?**

Imagina que você tem um funcionário super caro que faz um trabalho ótimo, mas tá pesando no bolso da empresa. Aí você descobre que tem outros candidatos que fazem o mesmo trabalho, às vezes até melhor, por um preço muito menor. É isso que a gente vai fazer aqui - só que em vez de funcionários, são modelos de IA! 😄

**Por que migração de modelo é importante?**

É como trocar de plano de celular: às vezes você paga caro por algo que nem usa tanto, e tem planos melhores por menos dinheiro. Com IA é a mesma coisa - modelos novos aparecem todo dia, e alguns são mais baratos e rápidos que os antigos.

---

**🖼️ Sugestão de imagem**: Um gráfico mostrando custos de diferentes modelos de IA

### **Nosso Cenário de Negócio**

Vamos simular uma situação real:

- **Problema**: Uma empresa usa um modelo proprietário caro pra fazer resumos de documentos
- **Dor**: O modelo é bom, mas tá custando uma fortuna pra rodar
- **Solução**: Vamos testar modelos mais baratos do Amazon Bedrock

### **Critérios de Sucesso**

É como quando você vai comprar um carro - você tem uma lista do que quer:

| Critério | O Que Queremos | Como Medir |
|----------|----------------|------------|
| **Qualidade** | Resumos tão bons quanto os atuais | IA avaliando os resumos |
| **Velocidade** | Pelo menos tão rápido quanto | Cronômetro nas respostas |
| **Custo** | Mais barato que o atual | Calculadora de preços |

Vamos começar explorando nossos dados e analisando como o modelo atual tá performando!

In [ ]:
# 🛠️ INSTALANDO AS FERRAMENTAS
!pip install --upgrade numexpr

### **Preparando o Dataset**

#### **Sobre o Dataset**

Um dataset de qualidade é fundamental pro sucesso da migração. É como ter uma régua boa pra medir - se a régua tá torta, todas as medidas vão dar errado!

Vamos usar o dataset [EdinburghNLP/xsum](https://huggingface.co/datasets/EdinburghNLP/xsum), que é tipo um "padrão ouro" pra resumos. Ele tem notícias da BBC com resumos escritos por humanos - perfeito pra testar se nossos modelos conseguem fazer resumos tão bons quanto pessoas reais.

Selecionamos 10 amostras representativas pra manter o workshop rápido e eficiente. Essas amostras representam vários tamanhos e níveis de complexidade pra dar uma avaliação robusta.

> **💡 Dica do Pedro**: Se você tá rodando isso no seu ambiente, pode substituir por seu próprio dataset. Só certifique que tem as colunas `document` e `referenceResponse`. Se quiser ver como preprocessamos o dataset, dá uma olhada no arquivo `src/dataset.py`.

Vamos dar uma olhada no que nossos dados parecem:

In [ ]:
# �� DANDO UMA OLHADA NOS DADOS
! head -5 ../data/document_sample_10.csv

### **Análise do Modelo Fonte**

#### **Medindo a Performance Atual**

> **💡 Nota**: Num cenário real de migração, você coletaria essas métricas do seu sistema de produção. Se você tá construindo uma aplicação nova, pode não ter um modelo fonte pra comparar.

Um passo crítico no processo de migração é estabelecer uma linha de base confiável pro seu modelo atual. É como marcar onde você tá antes de começar uma corrida - sem isso, como você vai saber se melhorou?

Pra esse workshop, já geramos respostas do nosso modelo fonte hipotético usando o dataset.

> **💡 Dica do Pedro**: Se quiser usar seu próprio modelo fonte (tipo GPT-4o-mini da OpenAI ou outro), pode gerar respostas seguindo o mesmo formato e substituir o arquivo CSV fornecido.

Vamos examinar uma das respostas do modelo fonte pra entender suas características de performance:

In [ ]:
# �� ANALISANDO O MODELO FONTE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurando o visual
plt.style.use('default')
sns.set_palette("husl")

# Carregando os dados do modelo fonte
source_model_data = pd.read_csv('../outputs/document_summarization_source_model.csv')

print("📊 ESTATÍSTICAS DO MODELO FONTE:")
print(f"�� Total de amostras: {len(source_model_data)}")
print(f"⏱️ Latência média: {source_model_data['latency'].mean():.3f} segundos")
print(f"💰 Custo médio: ${source_model_data['cost'].mean():.6f} por inferência")
print(f"🔤 Tokens de entrada médios: {source_model_data['model_input_tokens'].mean():.1f}")
print(f"🔤 Tokens de saída médios: {source_model_data['model_output_tokens'].mean():.1f}")

# Vamos dar uma olhada numa resposta específica
print("\n�� EXEMPLO DE RESPOSTA:")
sample_response = source_model_data.iloc[0]
print(f"Documento original: {sample_response['document'][:200]}...")
print(f"Resumo gerado: {sample_response['model_response']}")
print(f"Resumo de referência: {sample_response['referenceResponse']}")

### **Visualizando a Performance**

Vamos criar alguns gráficos pra entender melhor como o modelo fonte tá performando:

In [ ]:
# �� CRIANDO VISUALIZAÇÕES
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Gráfico 1: Distribuição de latência
axes[0, 0].hist(source_model_data['latency'], bins=20, alpha=0.7, color='skyblue')
axes[0, 0].set_title('⏱️ Distribuição de Latência')
axes[0, 0].set_xlabel('Latência (segundos)')
axes[0, 0].set_ylabel('Frequência')

# Gráfico 2: Distribuição de custos
axes[0, 1].hist(source_model_data['cost'], bins=20, alpha=0.7, color='lightgreen')
axes[0, 1].set_title('💰 Distribuição de Custos')
axes[0, 1].set_xlabel('Custo por inferência ($)')
axes[0, 1].set_ylabel('Frequência')

# Gráfico 3: Tokens de entrada vs saída
axes[1, 0].scatter(source_model_data['model_input_tokens'], 
                   source_model_data['model_output_tokens'], 
                   alpha=0.6, color='orange')
axes[1, 0].set_title('🔤 Tokens de Entrada vs Saída')
axes[1, 0].set_xlabel('Tokens de Entrada')
axes[1, 0].set_ylabel('Tokens de Saída')

# Gráfico 4: Latência vs custo
axes[1, 1].scatter(source_model_data['latency'], 
                   source_model_data['cost'], 
                   alpha=0.6, color='purple')
axes[1, 1].set_title('⏱️ Latência vs Custo')
axes[1, 1].set_xlabel('Latência (segundos)')
axes[1, 1].set_ylabel('Custo ($)')

plt.tight_layout()
plt.show()

print("\n💡 O que esses gráficos nos dizem?")
print("• A latência varia bastante - alguns documentos são mais complexos que outros")
print("• O custo tá relacionado com a quantidade de tokens")
print("• Documentos maiores geram resumos maiores (faz sentido!)")

### **Definindo Nossos Critérios de Sucesso**

Agora que entendemos como o modelo fonte tá performando, vamos definir o que significa "sucesso" pra nossa migração. É como definir as regras de um jogo - todo mundo precisa saber o que é vitória!

Vamos criar um dataframe de tracking que vai acompanhar todas as nossas avaliações:

In [ ]:
# �� CRIANDO NOSSO SISTEMA DE TRACKING
import pandas as pd
from datetime import datetime

# Definindo os modelos que vamos testar
models_to_evaluate = [
    'source_model',  # Nosso baseline
    'amazon.nova-lite-v1:0',  # Modelo da Amazon (barato e rápido)
    'us.anthropic.claude-3-5-haiku-20241022-v1:0'  # Claude Haiku (bom custo-benefício)
]

# Criando o dataframe de tracking
evaluation_tracking = pd.DataFrame({
    'model': models_to_evaluate,
    'region': 'us-east-1',  # Região da AWS
    'inference_profile': 'standard',  # Perfil de inferência
    'text_prompt': '',  # Vamos preencher isso no próximo passo
    'quality_evaluation_jobArn': '',  # Vamos preencher isso depois
    'quality_evaluation_output': ''  # Vamos preencher isso depois
})

print("📊 NOSSO PLANO DE AVALIAÇÃO:")
print(evaluation_tracking)

# Salvando o tracking
evaluation_tracking.to_csv('../data/evaluation_tracking.csv', index=False)
print("\n✅ Tracking salvo! Vamos usar isso pra acompanhar todo o processo.")

### **Resumo do Passo 1**

�� **Parabéns!** Você acabou de completar o primeiro passo da nossa jornada de migração. Vamos recapitular o que fizemos:

✅ **Entendemos o problema**: Modelo caro que precisa ser substituído
✅ **Analisamos os dados**: Dataset de qualidade pra testar
✅ **Estabelecemos baseline**: Performance atual do modelo fonte
✅ **Definimos critérios**: O que significa sucesso
✅ **Criamos tracking**: Sistema pra acompanhar tudo

### **O Que Vem no Próximo Passo**

No próximo notebook, vamos fazer algo super legal: **otimizar prompts**! É como ensinar cada modelo a falar da melhor forma possível. Diferentes modelos respondem melhor a diferentes tipos de instruções, então vamos usar o Amazon Bedrock Prompt Optimizer pra fazer essa mágica acontecer.

---

**�� Dica do Pedro**: Lembre-se, migração de modelo não é só trocar um pelo outro - é um processo cuidadoso que precisa de dados, métricas e muito teste!

**🚀 Próximo passo**: Otimização de prompts com Bedrock Prompt Optimizer